# RAG提取文中的氨基酸肽序列

In [1]:
import pandas as pd
import json

#获取文章的pmcid-pmid键值对
dic={'pmcid':[],'pmid':[]}
with open('/mnt/nas/handu/data/anti_inflammation_pubmed/full_text_pmcid_pmid_map.json','r') as f:
    js_list=json.load(f)
    for unit in js_list:
        dic['pmcid'].append(unit['pmcid'])
        dic['pmid'].append(unit['pmid'])

df_patent=pd.DataFrame(dic)

In [2]:
#获取review标签
with open('/mnt/nas/handu/data/anti_inflammation_pubmed/review_pmid_list.txt','r') as f:
    review_list=f.read().split('\n')
df_patent['is_review']=df_patent['pmid'].apply(lambda x: 1*(x in review_list))

In [3]:
df_patent

,pmcid,pmid,is_review
0,PMC6544583,31193882,0
1,PMC10409771,37553374,0
2,PMC4760456,26660549,0
3,PMC7429751,32795323,0
4,PMC8017164,33815353,0
...,...,...,...
2737,PMC9014016,35444957,0
2738,PMC4484568,26175842,0
2739,PMC4281220,25551573,0
2740,PMC4671727,26642048,0


## 商用大模型API加载

In [4]:
import os

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

In [5]:
from llama_index.llms.perplexity import Perplexity

pplx_api_key = "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"

llm = Perplexity(
    api_key=pplx_api_key, model="mixtral-8x7b-instruct", temperature=0.0,
    api_base="https://api.perplexity.ai",
)

In [6]:
from llama_index import SimpleDirectoryReader
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context,
)

from llama_index.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)

from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage import StorageContext
from llama_index.node_parser import get_leaf_nodes, get_root_nodes
from llama_index import VectorStoreIndex
from llama_index.retrievers.auto_merging_retriever import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

#载入模型以及embedding功能
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(
    model_name="/home/cymei/bge-small-en",device='cuda:3'
)

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model #"local:/home/cymei/bge-small-en"
)
set_global_service_context(service_context)



/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 混合索引引擎

In [7]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.schema import BaseNode

from typing import List

In [8]:
from nltk import word_tokenize 

class CustomRetriever(BaseRetriever):
    """自定义的混合索引类：相似度索引+关键词索引"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        vector_retriever_large: VectorIndexRetriever,
        max_num_keyword_nodes=3,
        keywords=[]
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever #头部相似度节点
        self._vector_retriever_large = vector_retriever_large #更大范围的头部相似度节点(用于关键词搜索)

        self.keywords=keywords #索引依据的关键词
        self.max_num_keyword_nodes=max_num_keyword_nodes #设置最大关键词节点数量
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        #利用两个不同的参数的retriever进行retrieve
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        vector_nodes_large = self._vector_retriever_large.retrieve(query_bundle)

        #确保集合中的节点id对应的相似度由大到小排列
        vector_ids = {n.node_id for n in sorted(vector_nodes,key=lambda node: node.score,reverse=True)} 
        # vector_ids_large = {n.node_id for n in vector_nodes_large}

        #对于更大范围的相似度索引结果，取出其中含有关键词的节点
        keyword_ids = []
        for n in sorted(vector_nodes_large,key=lambda node: node.score,reverse=True):
            for k in self.keywords:
                if(k in word_tokenize(n.get_content())):
                    #判断关键词是否在文章片段的分词结果中
                    keyword_ids.append(n.node_id)
                    break

        combined_dict = {n.node_id: n for n in vector_nodes}
        combined_dict.update({n.node_id: n for n in vector_nodes_large if n.node_id in keyword_ids })

        #合并两组节点
        if(keyword_ids==[]):
            #不含有关键词的情况下照常进行相似度索引
            retrieve_ids = vector_ids
        else:
            keyword_ids_top=set(keyword_ids[:self.max_num_keyword_nodes]) #取相似度最高的几个关键词节点
            vector_ids_unique=vector_ids-keyword_ids_top  #top相似度集合中独有的节点
            retrieve_ids=keyword_ids_top #关键词集合和top相似度集合共有的节点+关键词集合中独有的节点
            add_num=len(vector_ids)-len(keyword_ids_top)
            retrieve_ids=set(list(vector_ids_unique)[:add_num]).union(retrieve_ids) #额外添加部分top相似度集合中独有的节点

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [9]:
class LLM_summary:
    def __init__(self,documents=[],keywords=[]):
        self.keywords=keywords
        #载入专利文本
        self.documents=documents
    
    def hierarchy_node_index(self,chunk_list=[1024, 512, 256],similarity_top_k=6,max_num_keyword_nodes=3):
        '''
        获取分层节点，并建立对应索引
        设置每一层的token数量、选入prompt的节点数量以及最多含有关键词的节点数量
        '''
        node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_list)
        #获取分段后的每个节点
        self.nodes = node_parser.get_nodes_from_documents(self.documents)

        #建立各个节点的存储
        docstore = SimpleDocumentStore()
        # insert nodes into docstore
        docstore.add_documents(self.nodes)
        # define storage context (will include vector store by default too)
        storage_context = StorageContext.from_defaults(docstore=docstore)
        self.leaf_nodes = get_leaf_nodes(self.nodes)
    
        # 利用叶子节点计算相似度，并关联到对应的父节点
        base_index = VectorStoreIndex(
            self.leaf_nodes,
            storage_context=storage_context,
            #service_context=self.service_context,
        )

        #用于寻找和问题最相近的头部文本片段
        base_retriever = base_index.as_retriever(similarity_top_k=similarity_top_k)
        #用于从更大范围中寻找含有关键词的文本片段
        large_retriever = base_index.as_retriever(similarity_top_k=len(self.leaf_nodes))
        #构建混合索引
        custom_retriever = CustomRetriever(base_retriever, large_retriever,max_num_keyword_nodes=max_num_keyword_nodes,keywords=self.keywords)

        retriever = AutoMergingRetriever(custom_retriever, storage_context, verbose=True)

        return retriever 

## 批量文章挖掘seq

In [10]:
from llama_index.llms import ChatMessage

#设置system_prompt
system_str=  """You are a biologist and patent expert. 
    You will be provided with some contents from a patent and will be asked to answer specific questions related to the patent. 
    Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""

In [11]:
import json
import os
import re
from llama_index.schema import Document
#os.chdir('/home/cymei/nlp_task/anti_inflammation')

#专利文本载入函数。由于此次任务是寻找seq序列，和之前seq序列判断任务不同，暂时不考虑文本预处理
def load_doc(file_name):
    with open(file_name,'r') as f:
        js_file=json.load(f)
        js_text=js_file['text'] #只使用专利文件中的text内容

    return Document(text=js_text)

In [12]:
import os
from IPython.display import Markdown, display
import time

os.environ["http_proxy"] = "http://192.168.1.254:7890"
os.environ["https_proxy"] = "http://192.168.1.254:7890"

#从review和非review样本中随机各抽取5个进行尝试
df_test=pd.concat([df_patent[df_patent['is_review']==0].sample(5),df_patent[df_patent['is_review']==1].sample(5)]) 

#记录模型回答以及支持观点的段落
df_test['model_response']=''
df_test['supportive_context']=''

patent_list=df_test['pmcid'].unique()   #df_seq['pmcid'].unique()  PMC2408570  PMC6554247  PMC9786789  PMC6544583

for patent in patent_list:

    file_loc='/mnt/nas/handu/data/anti_inflammation_pubmed/pmc_parse/{}.json'.format(patent)
    doc_text=load_doc(file_loc)
    
    #使用llama-index建立查询引擎
    summarier=LLM_summary([doc_text])
    #分别从claim和des中进行检索
    # summarier_claim=LLM_summary([doc_dic['claim']])
    # summarier_des=LLM_summary([doc_dic['description']])

    #设置检索关键词
    summarier.keywords=['peptide','anti-inflammatory'] 

        # summarier_claim.keywords=[row['seq_no']]
        # summarier_des.keywords=[row['seq_no']]
    retriever=summarier.hierarchy_node_index(similarity_top_k=10,max_num_keyword_nodes=4)
        # retriever_claim=summarier_claim.hierarchy_node_index(similarity_top_k=2,max_num_keyword_nodes=2)
        # retriever_des=summarier_des.hierarchy_node_index(similarity_top_k=8,max_num_keyword_nodes=3)

    #query中的问题
    #目前该prompt可以引导模型在部分文字中抓取一些seq，但模型无法判断是完整的肽序列还是片段
    question="""Given content describing the functions and applications of the patent.
        Please find out all the peptides sequence firstly(motif sequence is not included). The target sequence should only consist of different kinds of peptide.
        Finally, plesae do analysis and explain the reason first, 
        then list all peptides sequences that have the ability of anti-inflammation line by line.
        Please focuse on the given context and don't use prior knowledge. Please return the top 3 most supportive context number.
            """
        #Make sure the target sequence numbers are specific about anti-inflammation and don't make any guesses.
        #Please focuse on the given context and don't use prior knowledge. Please return the most supportive context number.

    #retriever寻找目标节点
    response_nodes=retriever.retrieve(question)  

        #分别从claim和des中进行检索，确保两部分内容都会被用上
        # response_nodes_claim=retriever_claim.retrieve(question) 
        # response_nodes_des=retriever_des.retrieve(question) 
        # response_nodes=response_nodes_claim + response_nodes_des
    print(question)
    context_str="""Context is :
            {}
            ---------------------
            Given the provided context, please answer the query:
            {}
            """.format('\n'.join(['Context{}:\n'.format(index) + node.text for index,node in enumerate(response_nodes)]),question)  
    messages_dict = [
            {"role": "system", "content": system_str},
            {"role": "user", "content": context_str},
        ]
    messages = [ChatMessage(**msg) for msg in messages_dict]
    response = llm.chat(messages)

    display(Markdown(f"<b>{response.raw['usage']}</b>"))
    print('--------------------------------------------------------------------')
    display(Markdown(f"<b>{response}</b>"))

    #根据模型回答找出支持的文本片段
    response_ans=response.message.content
    context_list=re.findall('Context[0-9]+',response_ans)
    context_nums=set([int(re.sub('Context','',context)) for context in context_list])
    condition=(df_test['pmcid']==patent)

    df_test.loc[condition,'model_response']=response_ans
    df_test.loc[condition,'supportive_context']='\n'.join(['Context{}:\n'.format(index) + response_nodes[index].text for index in context_nums if(index<len(response_nodes))])
    time.sleep(30) #设置间隔防止api调用频率过高

> Merging 1 nodes into parent node.
> Parent node id: b2425a31-5b0b-4363-855b-a2032fc3987f.
> Parent node text: These results indicate that a slight drug association exists when the peptides are within the pol...

> Merging 2 nodes into parent node.
> Parent node id: 4f42d184-b618-482e-bd03-b5a963d7756e.
> Parent node text: Figure S4                  MALDI-TOF spectra of (A) Rho-POD and (B) Rho-HIV-Tat peptides.        ...

Given content describing the functions and applications of the patent.
        Please find out all the peptides sequence firstly(motif sequence is not included). The target sequence should only consist of different kinds of peptide.
        Finally, plesae do analysis and explain the reason first, 
        then list all peptides sequences that have the ability of anti-inflammation line by line.
        Please focuse on the given context and don't use prior knowledge. Please return the top 3 most supportive context number.
            


<b>{'prompt_tokens': 2492, 'completion_tokens': 374, 'total_tokens': 2866}</b>

--------------------------------------------------------------------


<b>assistant: After analyzing the given contexts, I have found the following peptide sequences:

1. POD (CGGG[ARKKAAKA]4)
2. HIV-Tat (CGGGGYGRKKRRQRRR)

Now, I will explain the reason for selecting these two sequences and why I believe they have the ability for anti-inflammation.

Context0 states that a "slight drug association exists when the peptides are within the polymer matrix" and different characteristics were observed depending on the preparation method. This implies that the peptides used in the study have some kind of therapeutic effect, possibly including anti-inflammatory properties.

In Context1, an anti-inflammatory efficacy assay is described, where inflammation is induced by topical administration of sodium arachidonate (SA) and the treatment protocol consists of instilling formulations after 30 minutes. The formulations include NPs with peptides, suggesting that the peptides might have an anti-inflammatory effect.

Context3 mentions that anti-inflammatory efficacy assays were performed to determine whether the novel formulations (PLGA-NPs-PEG-peptide and PLGA-PEG-peptide NPs) could form the basis of a suitable strategy for ocular delivery of NSAIDs, further implying that the peptides might have anti-inflammatory properties.

Based on the above analysis, I believe that both POD and HIV-Tat peptides have the ability for anti-inflammation.

Top 3 most supportive context numbers: 1, 3, and 0.</b>

> Merging 2 nodes into parent node.
> Parent node id: 74a05a2f-1beb-4fc2-95de-3f8245aece59.
> Parent node text: 1C,D,G,H). Further quantification of the area infiltrated with lymphocytes in the whole SMG showe...

> Merging 2 nodes into parent node.
> Parent node id: 6f56f46f-8d6a-45d5-bc66-81216ece52af.
> Parent node text: Our results indicated that the pro-inflammatory cytokines TNF-α, IL-6, IFN-β and IFN-γ were signi...

> Merging 2 nodes into parent node.
> Parent node id: 48ae593a-fa9c-4e93-b545-bed834f7692c.
> Parent node text: 1C,D,G,H). Further quantification of the area infiltrated with lymphocytes in the whole SMG showe...

Given content describing the functions and applications of the patent.
        Please find out all the peptides sequence firstly(motif sequence is not included). The target sequence should only consist of different kinds of peptide.
        Finally, plesae do analysis and explain the reason first, 
        then list all peptides sequences that have the abi

...

<b>assistant: Based on the given context, there are several peptides mentioned that are relevant to anti-inflammatory functions. Here are the peptides sequences and the reasons why they have the ability of anti-inflammation:

1. GLP-1 (glucagon-like peptide-1)
	* Sequence: not explicitly mentioned in the context, but it is a well-known 30-amino acid peptide hormone derived from the preproglucagon gene.
	* Anti-inflammatory function: GLP-1 has been shown to exert broad anti-inflammatory actions and reduce biomarkers of systemic inflammation in human subjects with type 2 diabetes and people with obesity (Context1). Preclinical studies also suggest that GLP-1RAs reduce cytokine production and attenuate pulmonary inflammation (Context1).
2. Exenatide
	* Sequence: Exenatide is a 39-amino acid peptide with the sequence: HGEGTFTSDLSKQMEEEAVRLFIEWLKNGGPDFCRDKLSTRKQQKVDDTTKGRKRYTMTSQTQRLNR (Context4).
	</b>

> Merging 2 nodes into parent node.
> Parent node id: 453c3828-e61d-4fce-a6b5-f6daf8ca5884.
> Parent node text: Interestingly, the anti-inflammatory effects rely mostly on suppressive action, while pro-resolvi...

> Merging 2 nodes into parent node.
> Parent node id: 4ef9681a-038b-4ed9-9b7a-e402e66a3b93.
> Parent node text: ANXA1 was reported to also be associated with the early stage of AD in patients and in animal mod...

> Merging 2 nodes into parent node.
> Parent node id: c420e97f-f2a7-4106-bc5e-0cf7ef437b54.
> Parent node text: ANXA1 was reported to also be associated with the early stage of AD in patients and in animal mod...

Given content describing the functions and applications of the patent.
        Please find out all the peptides sequence firstly(motif sequence is not included). The target sequence should only consist of different kinds of peptide.
        Finally, plesae do analysis and explain the reason first, 
        then list all peptides sequences that have the abi

<b>{'prompt_tokens': 2900, 'completion_tokens': 544, 'total_tokens': 3444}</b>

--------------------------------------------------------------------


<b>assistant: After reviewing the provided context, I have found several peptides that have the ability of anti-inflammation. Here are the analysis and the list of peptides:

Analysis:
According to Context0, ANXA1 peptides, such as Ac2–26 and Ac9–25, have the ability to fully activate FPR1 and induce pro-inflammatory response at high concentration. However, there is no mention of their anti-inflammatory effects. Therefore, we cannot include ANXA1 peptides in the list of anti-inflammatory peptides.

According to Context2, LXA4 has been shown to promote the release of anti-inflammatory factors and to exhibit the ability to reduce Aβ and phosphorylated tau levels. LXA4 binds to FPR2 receptor and activates many intracellular signaling pathways, leading to an increase in macrophages life span, inhibition of pro-inflammatory cytokines synthesis, and reduction of Aβ and phosphorylated tau levels. Therefore, LXA4 has the ability of anti-inflammation.

According to Context4, SPMs, including LXA4 and RvD1, activate cascades that induce remodeling within sites damaged by inflammatory processes. They promote several processes crucial for resolution of inflammation, such as neutrophil extravasation blockade, promotion non-phlogistic monocyte recruitment, suppression of proinflammatory mediators while potentiating anti-inflammatory cytokines release and macrophage phagocytosis and efferocytosis, altering macrophages phenotype and instructing cells to favor repair. Therefore, SPMs have the ability of anti-inflammation.

Based on the above analysis, I have found two peptides with the ability of anti-inflammation: LXA4 and RvD1.

List of peptides with anti-inflammatory effects:

1. LXA4
2. RvD1

Top 3 most supportive context numbers: Context2, Context4, Context5 (although Context5 does not directly mention LXA4 or RvD1, it supports the role of SPMs in anti-inflammation, which includes LXA4 and RvD1).</b>

In [13]:
df_test

,pmcid,pmid,is_review,model_response,supportive_context
1606,PMC4315550,25670897,0,"After analyzing the given contexts, I have fou...",Context0:\nThese results indicate that a sligh...
1911,PMC4827125,27064029,0,"Based on the provided context, there are no pe...",Context0:\nThe goal of this study was to deter...
257,PMC3232377,22059987,0,"Without the provided contents, I am unable to ...",
2024,PMC2902675,20631879,0,"After reviewing the provided contexts, I canno...",Context2:\nIn all of the experiments described...
425,PMC9686674,36421317,0,"Based on the provided context, here are the se...",Context0:\nIt was observed that WIKE-14 can ex...
2320,PMC10230051,37266425,1,"Based on the provided context, there are sever...",Context1:\nActivation of the GLP-1 pathways de...
793,PMC4216654,25338737,1,"Based on the given context, there are several ...",Context0:\n[42]Inhibts glucagon release by a m...
752,PMC8772713,35052724,1,"After analyzing the provided context, I have f...",Context1:\nThe LEAPS approach derives benefits...
1206,PMC9394722,36003669,1,"Based on the given context, there are several ...",Context1:\nThe so-called ‘Cytokine Storm’ that...
225,PMC8413167,34105114,1,"After reviewing the provided context, I have f...",Context0:\nIt seems that the dual properties m...


In [45]:
df_test.to_csv('./10patents_example.csv',index=False)

In [27]:
from llama_index.response.notebook_utils import display_source_node
for node in response_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** bc7b9bc5-089c-492d-b968-96c72a8e391f<br>**Similarity:** 0.8580143326929651<br>**Text:** In fact, a number of current TCM prescriptions use epimedium as the main ingredient or monarch such as the Xianling Gubao capsule, Er-Xian Decoction and Xian-Zhen Decoction. Other ingredients such as psoralea and drynaria rhizome serve as ministers or assistants to the monarch ingredient and are also included in these formulations.38      In this study, we present the development of a minimal TCM formula primarily containing two ingredients, epimedium and drynaria. Epimedium was chosen as the main or monarch ingredient partially because of its TCM warming properties but also because of its in vitro biological activities. Drynaria was chosen as a minister ingredient to the monarch to further strengthen the kidney and to support CRE activation and possible osteogenic activities. The other ingredients such as danshen were considered as assistants for invigorating the spleen and replenishment of qi as well as their anti-inflammatory activity.<br>

--------


**Node ID:** 67fb29a4-ade9-4616-95dd-e561678a8abb<br>**Similarity:** 0.8572586696340335<br>**Text:** LTD and E2 was from Bayer Pharmaceutical Co. Ltd. (Guangzhou, CN).        Vectors pGL4.29[luc2P/CRE/Hygro], pGL4.26 [luc2/minP/Hygro], pGL4.17 [luc2/Neo], pGL4.35 [luc2P/9XGAL4UAS/Hygro], pBIND-ERα-LBD (residues 299-595), and Fugene 6 and Fugene HD were purchased from Promega Inc. (Madison, WI). Vector pNF-κβ-Luc was from Clontech Laboratories (Mountain View, CA). Vectors pCMV6-XL4 ER β (Cat# SC119216), and pCMV6-NEO were purchased from Origene Technologies (Rockville, MD). Vector pCR BluntII TOPO ER α (Cat# MHS6278-211691051) was from Dharmacon (Lafayette, CO).<br>

--------


**Node ID:** 04a36885-efdb-4092-aa83-bf8bb589b0b1<br>**Similarity:** 0.8518454004028271<br>**Text:** 4                    3.3        Anti-inflammatory activity determination        Pretreatment of A549 cells containing the NF-κβ activation reporter construct with either danshen, epimedium, or milkvetch extracts (40 μg/mL) inhibited IL-1β (100 pg/mL) activation of NF-κβ (Fig. 5). The other extracts did not demonstrate appreciable anti-inflammatory activity. The anti-inflammatory activity did not correlate to CRE activation. Drynaria did not inhibit NF-κβ activation by IL-1β despite activating CRE. Bay 11-7082 (12.5 μM) was used as a control inhibitor of IL-1β activated NF-κβ.34Fig. 5Effect of selected TCM plant extracts on the inhibition of NF-κβ activation by IL-1β. Plant extracts were resuspended in DMSO and added at a final concentration of 40 μg/mL to A549 cells stably transfected with an NF-κβ responsive luciferase reporter construct. Cells were stimulated with IL-1β (100 pg/mL). Control treatments were IL-1β only and Bay11 (B; 12.5 μM). Cells were treated for 6 h. Extract abbreviations were as follows: An, Angelica; As, Asiatic Cornelian Cherry; D, Danshen; Dr, Drynaria; DS, Dodder Seed; E, Epimedium, Eu, Eucommia; GP, Glossy Privet; IB, Indian Bread; M, Milkvetch; MC, Medicinal Cyathula; P, Psoralea: R, Rehmannia; TA, Two-toothed Achyranthes; WB, Wolfberry root bark; and Y, Common Yan Rhizome. Data are represented as mean ± SD of relative luminescence units (RLUs (n = 3); #p < 0.0001 compared to untreated (Unt) controls).Fig. 5                    3.4        Activation of bone specific genes Runx2 and Bmp-4        For subsequent evaluations, epimedium and drynaria were selected as companion ingredients and tested in additional bone related assays. Epimedium was selected as the primary (or monarch) ingredient for four reasons: 1) it activates the full-length ERα and ERβ (Fig.<br>

...

--------


**Node ID:** 9a02eaa0-5f93-4579-a089-3ac3136e5230<br>**Similarity:** 0.8399784484138657<br>**Text:** Epimedium and drynaria activated these targets despite the presence of estrogenic materials, phenol-red and FBS.49 We suggest that epimedium and drynaria may be upregulating these genes through their activation of CRE. Fsk, a adenylate cyclase activator and epimedium have previously been shown to upregulate CRE-related responses.44 Drynaria has not been shown previously to activate CRE.      Osteoclast development was inhibited by the epimedium and drynaria combination in a dose-dependent manner and the addition of a third ingredient, either danshen, angelica or wolfberry augmented the inhibition at selected doses. Previously, epimedium alone has been shown to inhibit NF-κβ activation as well as to stimulate p38 MAPK to inhibit NF-κβ during osteoclast differentiation.45, 46, 47, 48 The NF-κβ inhibition by epimedium does not appear to be the result of either CRE activation and/or ER activation50, 51, 52 because other extracts that activate these pathways (drynaria and psoralea, respectively) do not inhibit NF-κβ activation.<br>

--------


**Node ID:** 2ebcb2ed-9bae-4a77-a7a7-3be231763a1f<br>**Similarity:** 0.8328065251198418<br>**Text:** The ERβ-LBD (residues 227-530) was PCR amplified from full-length human ERβ (SC119216, OriGene Technologies (Rockville, MD)) using forward and reverse primers 5′-CGATCGTTCGCCTTGTGCGGAGACAGAGAAGTGCC-3′ and 5′-TCTAGATTACTGAGACTGTGGGTTCTGGGAGCCCTC-3′, incorporating PvuI and XbaI sites at the 5′ and 3′ ends, respectively. The ERβ-LBD fragment was then ligated into the pBIND3 PvuI/XbaI site forming a Gal4- ERβ -LBD fusion protein.                            2.5        Cell culture and stable cell lines        Chinese hamster ovary (CHO-K1), MC3T3-E1 subclone 14, HEK293T/17, SK-N-SH, and A549 were purchased from ATCC (Manasas, VA) and grown in in DMEM or αMEM or DMEM or EMEM or F12K media, respectively.<br>

--------
